In [7]:
#Yasha Ali CT-22010
import math
import time

# Shared constants
HUMAN = 'X'
AI = 'O'
EMPTY = ' '

# Global node counters
minimax_nodes = 0
ab_nodes = 0

# Shared utility functions
def create_board():
    return [[EMPTY for _ in range(3)] for _ in range(3)]

def print_board(board):
    for row in board:
        print(" | ".join(row))
        print("-" * 9)

def is_full(board):
    return all(cell != EMPTY for row in board for cell in row)

def check_winner(board):
    lines = board + list(map(list, zip(*board)))  # rows + cols
    lines.append([board[i][i] for i in range(3)])  # main diagonal
    lines.append([board[i][2 - i] for i in range(3)])  # anti-diagonal
    for line in lines:
        if line == [HUMAN] * 3:
            return HUMAN
        elif line == [AI] * 3:
            return AI
    return None

def evaluate(board):
    winner = check_winner(board)
    if winner == AI:
        return 1
    elif winner == HUMAN:
        return -1
    return 0

def get_available_moves(board):
    return [(i, j) for i in range(3) for j in range(3) if board[i][j] == EMPTY]

# --------------------------
# MINIMAX (Without Pruning)
# --------------------------
def minimax(board, is_maximizing):
    global minimax_nodes
    minimax_nodes += 1
    winner = check_winner(board)
    if winner or is_full(board):
        return evaluate(board), None

    best_move = None
    if is_maximizing:
        max_eval = -math.inf
        for i, j in get_available_moves(board):
            board[i][j] = AI
            eval_score, _ = minimax(board, False)
            board[i][j] = EMPTY
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = (i, j)
        return max_eval, best_move
    else:
        min_eval = math.inf
        for i, j in get_available_moves(board):
            board[i][j] = HUMAN
            eval_score, _ = minimax(board, True)
            board[i][j] = EMPTY
            if eval_score < min_eval:
                min_eval = eval_score
                best_move = (i, j)
        return min_eval, best_move

def play_with_minimax():
    global minimax_nodes
    minimax_nodes = 0
    board = create_board()
    current_player = HUMAN
    print("\n--- Game 1: Minimax ---")
    print("You are X. AI is O.")
    print_board(board)

    start_time = time.time()

    while not check_winner(board) and not is_full(board):
        if current_player == HUMAN:
            while True:
                try:
                    move = input("Enter your move as row,col (0-2): ")
                    i, j = map(int, move.strip().split(','))
                    if board[i][j] == EMPTY:
                        board[i][j] = HUMAN
                        break
                    else:
                        print("Cell is already taken. Try again.")
                except:
                    print("Invalid input. Use format row,col (e.g., 1,2)")
        else:
            print("\nAI is thinking...")
            ai_start = time.time()
            _, move = minimax(board, True)
            duration = time.time() - ai_start
            print(f"AI chose: {move} (Minimax) in {duration:.4f}s, nodes: {minimax_nodes}")
            if move:
                board[move[0]][move[1]] = AI
        print_board(board)
        current_player = HUMAN if current_player == AI else AI

    game_duration = time.time() - start_time
    winner = check_winner(board)
    if winner == HUMAN:
        print("🎉 You win!")
    elif winner == AI:
        print("💻 AI wins!")
    else:
        print("It's a draw!")

    return minimax_nodes, game_duration

# --------------------------
# MINIMAX + ALPHA-BETA
# --------------------------
def minimax_ab(board, depth, alpha, beta, is_maximizing):
    global ab_nodes
    ab_nodes += 1
    winner = check_winner(board)
    if winner or is_full(board):
        return evaluate(board), None

    best_move = None
    if is_maximizing:
        max_eval = -math.inf
        for i, j in get_available_moves(board):
            board[i][j] = AI
            eval_score, _ = minimax_ab(board, depth + 1, alpha, beta, False)
            board[i][j] = EMPTY
            if eval_score > max_eval:
                max_eval = eval_score
                best_move = (i, j)
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = math.inf
        for i, j in get_available_moves(board):
            board[i][j] = HUMAN
            eval_score, _ = minimax_ab(board, depth + 1, alpha, beta, True)
            board[i][j] = EMPTY
            if eval_score < min_eval:
                min_eval = eval_score
                best_move = (i, j)
            beta = min(beta, eval_score)
            if beta <= alpha:
                break
        return min_eval, best_move

def play_with_alpha_beta():
    global ab_nodes
    ab_nodes = 0
    board = create_board()
    current_player = HUMAN
    print("\n--- Game 2: Alpha-Beta Pruning ---")
    print("You are X. AI is O.")
    print_board(board)

    start_time = time.time()

    while not check_winner(board) and not is_full(board):
        if current_player == HUMAN:
            while True:
                try:
                    move = input("Enter your move as row,col (0-2): ")
                    i, j = map(int, move.strip().split(','))
                    if board[i][j] == EMPTY:
                        board[i][j] = HUMAN
                        break
                    else:
                        print("Cell is already taken. Try again.")
                except:
                    print("Invalid input. Use format row,col (e.g., 1,2)")
        else:
            print("\nAI is thinking...")
            ai_start = time.time()
            _, move = minimax_ab(board, 0, -math.inf, math.inf, True)
            duration = time.time() - ai_start
            print(f"AI chose: {move} (Alpha-Beta) in {duration:.4f}s, nodes: {ab_nodes}")
            if move:
                board[move[0]][move[1]] = AI
        print_board(board)
        current_player = HUMAN if current_player == AI else AI

    game_duration = time.time() - start_time
    winner = check_winner(board)
    if winner == HUMAN:
        print("🎉 You win!")
    elif winner == AI:
        print("💻 AI wins!")
    else:
        print("It's a draw!")

    return ab_nodes, game_duration

# --------------------------
# Comparison Function
# --------------------------
def compare():
    mini_nodes, mini_time = play_with_minimax()
    ab_nodes, ab_time = play_with_alpha_beta()

    print("\n--- Performance Comparison ---")
    print(f"Standard Minimax: {mini_nodes} nodes, {mini_time:.4f}s total")
    print(f"Alpha-Beta Pruning: {ab_nodes} nodes, {ab_time:.4f}s total")

    if ab_nodes < mini_nodes:
        print("✅ Alpha-Beta Pruning was more efficient!")
    else:
        print("🤔 Alpha-Beta didn't reduce node count much this time.")

# Run comparison
if __name__ == "__main__":
    compare()



--- Game 1: Minimax ---
You are X. AI is O.
  |   |  
---------
  |   |  
---------
  |   |  
---------


Enter your move as row,col (0-2):  0,0


X |   |  
---------
  |   |  
---------
  |   |  
---------

AI is thinking...
AI chose: (1, 1) (Minimax) in 1.1426s, nodes: 59705
X |   |  
---------
  | O |  
---------
  |   |  
---------


Enter your move as row,col (0-2):  1,1


Cell is already taken. Try again.


...

O |   |  
---------
X | X |  
---------
  |   |  
---------

AI is thinking...
AI chose: (1, 2) (Alpha-Beta) in 0.0030s, nodes: 2495
O |   |  
---------
X | X | O
---------
  |   |  
---------


Enter your move as row,col (0-2):  2,0


O |   |  
---------
X | X | O
---------
X |   |  
---------

AI is thinking...
AI chose: (0, 2) (Alpha-Beta) in 0.0000s, nodes: 2522
O |   | O
---------
X | X | O
---------
X |   |  
---------


Enter your move as row,col (0-2):  2,2


O |   | O
---------
X | X | O
---------
X |   | X
---------

AI is thinking...
AI chose: (0, 1) (Alpha-Beta) in 0.0000s, nodes: 2526
O | O | O
---------
X | X | O
---------
X |   | X
---------
💻 AI wins!

--- Performance Comparison ---
Standard Minimax: 60684 nodes, 28.7504s total
Alpha-Beta Pruning: 2526 nodes, 27.6334s total
✅ Alpha-Beta Pruning was more efficient!


In [ ]:
2,2
